One of the most common probability distributions is the normal (or Gaussian) distribution.  Many natural phenomena can be modelled using a normal distribution.  It's also of great importance due to its relation to the [Central Limit Theorem](https://en.wikipedia.org/wiki/Central_limit_theorem).

In this post, we'll be reviewing the normal distribution and looking at how to draw samples from it using two methods.  One naive method using the central limit theorem, and another using the [Box-Muller transform](https://en.wikipedia.org/wiki/Box%E2%80%93Muller_transform).  As usual, some brief coverage of proofs and code will be included to help drive intuition.


## Background

Let's first start off covering some basics.  A normal distribution \\(N \sim (\mu, \sigma)\\) has probability density function (PMF) and cumulative density function (CDF) shown here [<sup>[1]</sup>](#fn-1):

$$ f_N(x) = \frac{1}{\sigma \sqrt{2\pi}}e^{-\frac{(x-\mu)^2}{2\sigma}}    $$
$$ F_N(x) = \int_{-\infty}^{x}f_N(t) dt $$

The CDF doesn't have a nice closed form, so we'll just represent it here using the definition of CDF in terms of its PDF.  We can graph the PDF and CDF (images from [Wikipedia](https://en.wikipedia.org/wiki/Normal_distribution)):


![alt text](https://github.com/adam-p/markdown-here/raw/master/src/common/images/icon48.png "Logo Title Text 1")



## Notes

List of Notes: [^1] 

[^1]: We'll use the convention of \\(f_X(x)\\) and \\(F_X(x)\\) to denote the PDF and CDF of random variable X, respectively.

